# Jydske Vestkysten

First, getting libraries and chrome

In [ ]:

!apt-get update
!apt-get install -y wget unzip libvulkan1


!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install -f -y

!apt-get install -y xvfb

!pip install selenium chromedriver-autoinstaller

import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from bs4 import BeautifulSoup



Get:1 https://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy Release
Get:6 https://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1,083 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Fetched 260 kB in 5s (50.7 kB/s)
Reading package lists...

In [ ]:



chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-images")
chrome_options.add_argument("--blink-settings=imagesEnabled=false")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")

#  WebDriver
driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 20)

# load  articles by clicking the vis flere btn
def load_all_articles():
    try:
        while True:
            show_more_button = wait.until(EC.element_to_be_clickable((By.ID, "show-more")))
            driver.execute_script("arguments[0].click();", show_more_button)
            time.sleep(2)
            print("Loaded more articles...")
    except TimeoutException:
        print("No more 'Vis flere' button detected.")

# extract article links from results
def get_article_links():
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    articles = soup.find_all('article', class_='search-result')
    article_links = []
    for article in articles:
        href = article.find('a')['href']
        full_url = "https://jv.dk" + href
        article_links.append(full_url)
    return article_links

# handle login if paywall appears
def handle_login():
    try:
        login_button = driver.find_element(By.CSS_SELECTOR, 'a#zephr-action-btn')
        login_button.click()
        print("Clicked login button, entering credentials...")

        email_input = wait.until(EC.element_to_be_clickable((By.ID, 'email')))
        password_input = wait.until(EC.element_to_be_clickable((By.ID, 'password')))
        email_input.send_keys('grp613@alumni.ku.dk')
        password_input.send_keys('xxxx')

        submit_button = driver.find_element(By.CSS_SELECTOR, 'button[type="submit"]')
        driver.execute_script("arguments[0].click();", submit_button)
        time.sleep(5)  # Wait for login to complete
        print("Logged in successfully!")
    except Exception as e:
        print("Login failed:", e)

# scrape article
def scrape_article(article_url, search_term, retries=3):
    for attempt in range(retries):
        try:
            print(f"Scraping article: {article_url} (Attempt {attempt + 1}/{retries})")
            driver.get(article_url)
            time.sleep(3)

            try:
                paywall = driver.find_element(By.CSS_SELECTOR, '.paywall-class')  # Adjust selector as needed
                if paywall:
                    print("Paywall detected, attempting to log in...")
                    handle_login()
                    driver.get(article_url)  # Reload the article after login
                    time.sleep(3)
            except NoSuchElementException:
                print("No paywall detected.")

            soup = BeautifulSoup(driver.page_source, 'html.parser')

            try:
                title = soup.find('h1', class_='article__headline').get_text(strip=True)
            except:
                title = 'No title'

            try:
                subheader = soup.find('div', class_='article__lead lead').get_text(strip=True)
            except:
                subheader = 'No subheader'

            try:
                date = soup.find('time', class_='article__date').get_text(strip=True)
            except:
                date = 'No date'

            try:
                content_divs = soup.find_all('div', class_='article__text')
                paragraphs = []

                for div in content_divs:
                    p_elements = div.find_all('p')
                    for p in p_elements:
                        paragraphs.append(p.get_text(strip=True))

                content = " ".join(paragraphs)
            except:
                content = 'No content'

            return {
                'url': article_url,
                'title': title,
                'subheader': subheader,
                'date': date,
                'content': content,
                'search_term': search_term
            }
        except TimeoutException:
            print(f"TimeoutException occurred on article: {article_url}. Retrying...")

    print(f"Failed to scrape article after {retries} attempts: {article_url}")
    return None


def scrape_articles(search_term):
    search_url = f"https://jv.dk/soeg?keywords={search_term}"
    driver.get(search_url)
    time.sleep(3)


    load_all_articles()

    article_links = get_article_links()

    all_articles = []
    for i, article_url in enumerate(article_links):
        article_data = scrape_article(article_url, search_term)
        if article_data:
            all_articles.append(article_data)

    return all_articles

# searcch terms
search_terms = ['ansigtsgenkendelse', 'facial recognition', 'face recognition', 'ansigtsaftryk', 'ansigtsscan',
                'masseovervågning ansigt', 'masseovervågning kamera', 'ansigtskamera', 'overvågningssamfund ansigt',
                'overvågningssamfund kamera', 'sikkerhedspakke overvågning', 'tryghedsskabende kamera']

all_scraped_articles = []

# Looping though search terms and scrape articles
for term in search_terms:
    print(f"Scraping articles for search term: {term}")
    scraped_articles = scrape_articles(term)
    all_scraped_articles.extend(scraped_articles)

df = pd.DataFrame(all_scraped_articles)
df.to_csv('jv_scraped_articles_multiple_terms.csv', index=False)
print(df.head())

driver.quit()


Scraping articles for search term: ansigtsgenkendelse
Loaded more articles...
Loaded more articles...
Loaded more articles...
Loaded more articles...
Loaded more articles...
No more 'Vis flere' button detected.
Scraping article: https://jv.dk/danmark/rigspolitiet-mangler-risikovurdering-af-ansigtsgenkendelse (Attempt 1/3)
No paywall detected.
Scraping article: https://jv.dk/nordtyskland/kamp-mod-islamistisk-terror-nu-bliver-automatisk-ansigtsgenkendelse-lovlig (Attempt 1/3)
No paywall detected.
Scraping article: https://jv.dk/danmark/fodboldklubber-vil-bruge-ansigtsgenkendelse-til-at-stoppe-voldsmaend (Attempt 1/3)
No paywall detected.
Scraping article: https://jv.dk/indland/datatilsyn-afkraever-svar-fra-politiet-om-brug-af-ansigtsgenkendelse (Attempt 1/3)
No paywall detected.
Scraping article: https://jv.dk/debat/regeringen-oensker-at-forvandle-danmark-til-et-overvaagningssamfund (Attempt 1/3)
No paywall detected.
Scraping article: https://jv.dk/danmark/folketinget-tillader-brug-af-an

In [ ]:
df['merged_content'] = df['title'] + ' ' + df['subheader'] + ' ' + df['content']
df

,url,title,subheader,date,content,search_term,merged_content
0,https://jv.dk/danmark/rigspolitiet-mangler-ris...,Rigspolitiet mangler risikovurdering af ansigt...,"Rigspolitiet kan endnu ikke give garantier, de...",21 okt. 2024 kl. 12:00,I september fik Rigspolitiet grønt lys af Folk...,ansigtsgenkendelse,Rigspolitiet mangler risikovurdering af ansigt...
1,https://jv.dk/nordtyskland/kamp-mod-islamistis...,Kamp mod islamistisk terror: Nu bliver automat...,,17 sep. 2024 kl. 18:22,KIEL: Efter en formodet islamistisk terrorist ...,ansigtsgenkendelse,Kamp mod islamistisk terror: Nu bliver automat...
2,https://jv.dk/danmark/fodboldklubber-vil-bruge...,Fodboldklubber vil bruge ansigtsgenkendelse ti...,Efter et møde hos justitsministeren betoner fo...,12 sep. 2024 kl. 16:35,,ansigtsgenkendelse,Fodboldklubber vil bruge ansigtsgenkendelse ti...
3,https://jv.dk/indland/datatilsyn-afkraever-sva...,Datatilsyn afkræver svar fra politiet om brug ...,Politiet har for nylig fået tilladelse til at ...,09 sep. 2024 kl. 17:25,Datatilsynet har sendt en række spørgsmål til ...,ansigtsgenkendelse,Datatilsyn afkræver svar fra politiet om brug ...
4,https://jv.dk/debat/regeringen-oensker-at-forv...,Regeringen ønsker at forvandle Danmark til et ...,,08 sep. 2024 kl. 14:33,"Læserbrev: I kampen mod svenske mindreårige, s...",ansigtsgenkendelse,Regeringen ønsker at forvandle Danmark til et ...
5,https://jv.dk/danmark/folketinget-tillader-bru...,Folketinget tillader politiet brug af ansigtsg...,"Regeringen og folketingspartier har aftalt, at...",05 sep. 2024 kl. 13:10,Politiet får nu tilladelse til at bruge ansigt...,ansigtsgenkendelse,Folketinget tillader politiet brug af ansigtsg...
6,https://jv.dk/udland/hollandsk-vagthund-idoemm...,Hollandsk vagthund idømmer ansigtsgenkendelses...,Amerikansk firma får endnu en trecifret millio...,03 sep. 2024 kl. 17:00,Hollandske myndigheder stempler tirsdag omfatt...,ansigtsgenkendelse,Hollandsk vagthund idømmer ansigtsgenkendelses...
7,https://jv.dk/danmark/el-bakker-op-om-brug-af-...,EL bakker op om brug af ansigtsgenkendelse i s...,Brug af ansigtsgenkendelse i sager om farlig b...,31 aug. 2024 kl. 23:40,Det kan spare tid og kræfter i et presset poli...,ansigtsgenkendelse,EL bakker op om brug af ansigtsgenkendelse i s...
8,https://jv.dk/indland/politiet-tager-hul-paa-f...,Politiet tager hul på første projekt med ansig...,Rigspolitiet har hentet tilbud på software til...,31 aug. 2024 kl. 16:10,"Politiet bekræfter, at man er i gang med det f...",ansigtsgenkendelse,Politiet tager hul på første projekt med ansig...
9,https://jv.dk/indland/dommere-vil-have-snor-i-...,Dommere vil have snor i politiets brug af ansi...,Brug af nyt værktøj er ligeså alvorligt et ind...,26 aug. 2024 kl. 08:15,Hvis politiet bruger ansigtsgenkendelse i efte...,ansigtsgenkendelse,Dommere vil have snor i politiets brug af ansi...


In [ ]:


# changing date
month_mapping = {
    'jan.': 'January', 'feb.': 'February', 'mar.': 'March',
    'apr.': 'April', 'maj': 'May', 'jun.': 'June',
    'jul.': 'July', 'aug.': 'August', 'sep.': 'September',
    'okt.': 'October', 'nov.': 'November', 'dec.': 'December'
}

def parse_date(date_string):
    try:
        date_part = date_string.split('kl.')[0].strip()

        for danish_month, english_month in month_mapping.items():
            if danish_month in date_part:
                date_part = date_part.replace(danish_month, english_month)

        parsed_date = pd.to_datetime(date_part, format='%d %B %Y', errors='coerce', dayfirst=True)
        return parsed_date.strftime('%Y-%m-%d') if parsed_date is not pd.NaT else None
    except Exception as e:
        return None

df['date'] = df['date'].apply(parse_date)
df



,url,title,subheader,date,content,search_term,merged_content,formatted_date
0,https://jv.dk/danmark/rigspolitiet-mangler-ris...,Rigspolitiet mangler risikovurdering af ansigt...,"Rigspolitiet kan endnu ikke give garantier, de...",2024-10-21,I september fik Rigspolitiet grønt lys af Folk...,ansigtsgenkendelse,Rigspolitiet mangler risikovurdering af ansigt...,2024-10-21
1,https://jv.dk/nordtyskland/kamp-mod-islamistis...,Kamp mod islamistisk terror: Nu bliver automat...,,2024-09-17,KIEL: Efter en formodet islamistisk terrorist ...,ansigtsgenkendelse,Kamp mod islamistisk terror: Nu bliver automat...,2024-09-17
2,https://jv.dk/danmark/fodboldklubber-vil-bruge...,Fodboldklubber vil bruge ansigtsgenkendelse ti...,Efter et møde hos justitsministeren betoner fo...,2024-09-12,,ansigtsgenkendelse,Fodboldklubber vil bruge ansigtsgenkendelse ti...,2024-09-12
3,https://jv.dk/indland/datatilsyn-afkraever-sva...,Datatilsyn afkræver svar fra politiet om brug ...,Politiet har for nylig fået tilladelse til at ...,2024-09-09,Datatilsynet har sendt en række spørgsmål til ...,ansigtsgenkendelse,Datatilsyn afkræver svar fra politiet om brug ...,2024-09-09
4,https://jv.dk/debat/regeringen-oensker-at-forv...,Regeringen ønsker at forvandle Danmark til et ...,,2024-09-08,"Læserbrev: I kampen mod svenske mindreårige, s...",ansigtsgenkendelse,Regeringen ønsker at forvandle Danmark til et ...,2024-09-08
5,https://jv.dk/danmark/folketinget-tillader-bru...,Folketinget tillader politiet brug af ansigtsg...,"Regeringen og folketingspartier har aftalt, at...",2024-09-05,Politiet får nu tilladelse til at bruge ansigt...,ansigtsgenkendelse,Folketinget tillader politiet brug af ansigtsg...,2024-09-05
6,https://jv.dk/udland/hollandsk-vagthund-idoemm...,Hollandsk vagthund idømmer ansigtsgenkendelses...,Amerikansk firma får endnu en trecifret millio...,2024-09-03,Hollandske myndigheder stempler tirsdag omfatt...,ansigtsgenkendelse,Hollandsk vagthund idømmer ansigtsgenkendelses...,2024-09-03
7,https://jv.dk/danmark/el-bakker-op-om-brug-af-...,EL bakker op om brug af ansigtsgenkendelse i s...,Brug af ansigtsgenkendelse i sager om farlig b...,2024-08-31,Det kan spare tid og kræfter i et presset poli...,ansigtsgenkendelse,EL bakker op om brug af ansigtsgenkendelse i s...,2024-08-31
8,https://jv.dk/indland/politiet-tager-hul-paa-f...,Politiet tager hul på første projekt med ansig...,Rigspolitiet har hentet tilbud på software til...,2024-08-31,"Politiet bekræfter, at man er i gang med det f...",ansigtsgenkendelse,Politiet tager hul på første projekt med ansig...,2024-08-31
9,https://jv.dk/indland/dommere-vil-have-snor-i-...,Dommere vil have snor i politiets brug af ansi...,Brug af nyt værktøj er ligeså alvorligt et ind...,2024-08-26,Hvis politiet bruger ansigtsgenkendelse i efte...,ansigtsgenkendelse,Dommere vil have snor i politiets brug af ansi...,2024-08-26


In [ ]:
df = df.drop(columns=['formatted_date'])

In [ ]:
df= df.rename(columns={'title': 'header'})
df

,url,header,subheader,date,content,search_term,merged_content
0,https://jv.dk/danmark/rigspolitiet-mangler-ris...,Rigspolitiet mangler risikovurdering af ansigt...,"Rigspolitiet kan endnu ikke give garantier, de...",2024-10-21,I september fik Rigspolitiet grønt lys af Folk...,ansigtsgenkendelse,Rigspolitiet mangler risikovurdering af ansigt...
1,https://jv.dk/nordtyskland/kamp-mod-islamistis...,Kamp mod islamistisk terror: Nu bliver automat...,,2024-09-17,KIEL: Efter en formodet islamistisk terrorist ...,ansigtsgenkendelse,Kamp mod islamistisk terror: Nu bliver automat...
2,https://jv.dk/danmark/fodboldklubber-vil-bruge...,Fodboldklubber vil bruge ansigtsgenkendelse ti...,Efter et møde hos justitsministeren betoner fo...,2024-09-12,,ansigtsgenkendelse,Fodboldklubber vil bruge ansigtsgenkendelse ti...
3,https://jv.dk/indland/datatilsyn-afkraever-sva...,Datatilsyn afkræver svar fra politiet om brug ...,Politiet har for nylig fået tilladelse til at ...,2024-09-09,Datatilsynet har sendt en række spørgsmål til ...,ansigtsgenkendelse,Datatilsyn afkræver svar fra politiet om brug ...
4,https://jv.dk/debat/regeringen-oensker-at-forv...,Regeringen ønsker at forvandle Danmark til et ...,,2024-09-08,"Læserbrev: I kampen mod svenske mindreårige, s...",ansigtsgenkendelse,Regeringen ønsker at forvandle Danmark til et ...
5,https://jv.dk/danmark/folketinget-tillader-bru...,Folketinget tillader politiet brug af ansigtsg...,"Regeringen og folketingspartier har aftalt, at...",2024-09-05,Politiet får nu tilladelse til at bruge ansigt...,ansigtsgenkendelse,Folketinget tillader politiet brug af ansigtsg...
6,https://jv.dk/udland/hollandsk-vagthund-idoemm...,Hollandsk vagthund idømmer ansigtsgenkendelses...,Amerikansk firma får endnu en trecifret millio...,2024-09-03,Hollandske myndigheder stempler tirsdag omfatt...,ansigtsgenkendelse,Hollandsk vagthund idømmer ansigtsgenkendelses...
7,https://jv.dk/danmark/el-bakker-op-om-brug-af-...,EL bakker op om brug af ansigtsgenkendelse i s...,Brug af ansigtsgenkendelse i sager om farlig b...,2024-08-31,Det kan spare tid og kræfter i et presset poli...,ansigtsgenkendelse,EL bakker op om brug af ansigtsgenkendelse i s...
8,https://jv.dk/indland/politiet-tager-hul-paa-f...,Politiet tager hul på første projekt med ansig...,Rigspolitiet har hentet tilbud på software til...,2024-08-31,"Politiet bekræfter, at man er i gang med det f...",ansigtsgenkendelse,Politiet tager hul på første projekt med ansig...
9,https://jv.dk/indland/dommere-vil-have-snor-i-...,Dommere vil have snor i politiets brug af ansi...,Brug af nyt værktøj er ligeså alvorligt et ind...,2024-08-26,Hvis politiet bruger ansigtsgenkendelse i efte...,ansigtsgenkendelse,Dommere vil have snor i politiets brug af ansi...


In [ ]:
from google.colab import files

df.to_csv('JV.csv', index=False)
files.download('JV.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>